In [1]:
# Initial imports
import numpy as np
import pandas as pd 
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

import random
import urllib.request
import requests
import shutil
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [ ]:
base_url = "https://memegenerator.net/memes/popular/alltime/page/"

columns = ['Title', 'Link', 'Image']
data = DataFrame(columns=columns)

for page in range(1, 201):
    if page % 20 == 0:
        print(page)
    url = base_url + str(page)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    img_divs = soup.findAll('div', {'class': 'char-img'})
    for img_div in img_divs:
        img_src = img_div.find('img').get('src')
        link = img_div.find('a').get('href')
        title = img_div.find('div', {'class': 'left'}).text.strip()
        tmp = DataFrame([[title, link, img_src]])
        tmp.columns = columns
        data = data.append(tmp, ignore_index=True)

In [ ]:
data.to_csv('../data/meme_metadata.csv', encoding='utf-8', index=False)

In [2]:
data = pd.read_csv('../data/meme_metadata.csv')

In [3]:
# EDIT THESE NUMBERS DEPENDING ON THE MEMES YOU ARE SUPPOSED TO CRAWL
start = 0 + 300
end = start + 100
data = data[start:end]

In [4]:
def download_web_image(url, name):
    r = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})
    full_name = '../data/images' + str(name) + '.jpg'
    
    if r.status_code == 200:
        with open(full_name, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)
    else:
        print('Could not download ' + str(name))
    return full_name

In [ ]:
columns = ['Title', 'Link', 'CaptionTop', 'CaptionBottom', 'ImageName', 'Upvotes']
caption_data = DataFrame(columns=columns)

for index, row in data.iterrows():
    if index % 500 == 0:
        print(index)
    meme_link = row['Link']
    img_link = row['Image']
    img_name = row['Title']
    download_name = download_web_image(img_link, meme_link)
    base_url = 'https://memegenerator.net'
    page = 1
    flag = True
    for page in range(1, 500):
        url = base_url + row['Link'] + '/images/popular/alltime/page/' + str(page)
        source_code = requests.get(url)
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text)
        for meme in soup.findAll('div', {'class': 'gallery-img'}):
            caption = meme.find('div', {'class': 'optimized-instance-container'})
            text0 = caption.find('div', {'class': 'optimized-instance-text0'}).text.strip()
            text1 = caption.find('div', {'class': 'optimized-instance-text1'}).text.strip()
            upvotes = int(meme.find('div', {'class': 'score'}).text.replace(',', '').strip())
            if upvotes < 0:
                continue
            tmp = DataFrame([[img_name, meme_link, text0, text1, download_name, upvotes]])
            tmp.columns = columns
            caption_data = caption_data.append(tmp, ignore_index=True)

In [ ]:
caption_data.to_csv('../data/caption_data_' + str(start) + '_' + str(end) + '.csv', encoding='utf-8', index=False)